In [1]:
import pandas as pd
import json
import os

folder_path = "Sample Data"
file_name = "orders.csv"
file_path = os.path.join(folder_path, file_name)

with open(file_path, "r", encoding="utf-8") as f:
    df = pd.read_csv(f)
print(df)
# Convert column names dynamically
df.columns = (
    df.columns.str.strip()  # Remove leading/trailing spaces
    .str.lower()            # Convert to lowercase
    .str.replace(" ", "_")  # Replace spaces with underscores
    .str.replace(r"[^\w\s]", "", regex=True)  # Remove special characters
)

      Order ID        Date  Customer ID  Product ID  Quantity  Total Price  \
0       764380  2024-01-01           25           6         3      1873.53   
1       879729  2024-01-01           46          18         5     10431.50   
2       134106  2024-01-01           22           1         3      6253.98   
3       926818  2024-01-01           93          20         2       889.48   
4       141647  2024-01-01           13          12         5     10995.65   
...        ...         ...          ...         ...       ...          ...   
2663    900230  2025-02-10           27           9         2      4221.38   
2664    795612  2025-02-10           90          13         2      1851.38   
2665    420965  2025-02-10           46           6         4      2498.04   
2666    539693  2025-02-10           98           3         2       698.44   
2667    448311  2025-02-10           78          18         1      2086.30   

       Profit  
0      526.21  
1     1295.49  
2      737.54  

In [3]:
import snowflake.connector
import os
import requests
PASSWORD = os.getenv('SNOWSQL_PWD')
print(PASSWORD)

5eWxWv4EvyCkhkY


In [7]:
try:
    ctx = snowflake.connector.connect(
        user='ABHINAVSHARMA2002',
        password=PASSWORD,
        account='qraojwa-yg67137'
    )
    cs = ctx.cursor()
    try:
        cs.execute("CREATE WAREHOUSE IF NOT EXISTS task_1_warehouse_mg")
        cs.execute("CREATE DATABASE IF NOT EXISTS testdb_mg")
        cs.execute("USE DATABASE testdb_mg")
        cs.execute("CREATE SCHEMA IF NOT EXISTS task_1_mg")
        cs.execute("USE WAREHOUSE task_1_warehouse_mg")
        cs.execute("USE SCHEMA task_1_mg")
        runQueries(cs,df)
    except Exception as e:
        print(f"Error: {e}") 
    finally:
        cs.close()
        ctx.close()
except Exception as e:
    print(f"Error: {e}")

In [5]:
def runQueries(cs,df):    
    # Rename specific columns
    #df["customer_id"] = df["customer_id"].astype(str)
    df = df.astype(str).apply(lambda col: col.map(lambda x: None if pd.isna(x) or x == "" or str(x).strip().lower() == "nan" else x))

    # Generate CREATE TABLE statement dynamically
    table_name = "article_table_pandas_4"
    columns = ", ".join([f'{col} STRING' for col in df.columns])
    create_table_sql = f'CREATE OR REPLACE TABLE {table_name} ({columns})'
    cs.execute(create_table_sql)

    # Insert data into Snowflake
    for _, row in df.iterrows():
        placeholders = ", ".join(["%s"] * len(row))
        insert_sql = f'INSERT INTO {table_name} ({", ".join(df.columns)}) VALUES ({placeholders})'
        cs.execute(insert_sql, tuple(row))